In [1]:
%run ./tutorials/wikiqa/init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
import numpy as np
import pandas as pd
from keras.optimizers import Adam
from keras.utils import to_categorical

import matchzoo as mz
from matchzoo.contrib.models.esim import ESIM

In [3]:
def load_filtered_data(preprocessor, data_type):
    assert ( data_type in ['train', 'dev', 'test'])
    data_pack = mz.datasets.wiki_qa.load_data(data_type, task='ranking')

    if data_type == 'train':
        X, Y = preprocessor.fit_transform(data_pack).unpack()
    else:
        X, Y = preprocessor.transform(data_pack).unpack()

    new_idx = []
    for i in range(Y.shape[0]):
        if X["length_left"][i] == 0 or X["length_right"][i] == 0:
            continue
        new_idx.append(i)
    new_idx = np.array(new_idx)
    print("Removed empty data. Found ", (Y.shape[0] - new_idx.shape[0]))

    for k in X.keys():
        X[k] = X[k][new_idx]
    Y = Y[new_idx]

    pos_idx = (Y == 1)[:, 0]
    pos_qid = X["id_left"][pos_idx]
    keep_idx_bool = np.array([ qid in pos_qid for qid in X["id_left"]])
    keep_idx = np.arange(keep_idx_bool.shape[0])
    keep_idx = keep_idx[keep_idx_bool]
    print("Removed questions with no pos label. Found ", (keep_idx_bool == 0).sum())

    print("shuffling...")
    np.random.shuffle(keep_idx)
    for k in X.keys():
        X[k] = X[k][keep_idx]
    Y = Y[keep_idx]

    return X, Y, preprocessor

In [4]:
fixed_length_left = 10
fixed_length_right = 40
batch_size = 32
epochs = 5

In [5]:
# prepare data
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=fixed_length_left,
                                                  fixed_length_right=fixed_length_right,
                                                  remove_stop_words=False,
                                                  filter_low_freq=10)

train_X, train_Y, preprocessor = load_filtered_data(preprocessor, 'train')
val_X, val_Y, _ = load_filtered_data(preprocessor, 'dev')
pred_X, pred_Y = val_X, val_Y
# pred_X, pred_Y, _ = load_filtered_data(preprocessor, 'test') # no prediction label for quora dataset

embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'], initializer=lambda: 0)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10798.93it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8019.65it/s]
Building Vocabulary from a datapack.: 100%|██████████| 358408/358408 [00:00<00:00, 4845654.07it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 15108.05it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8129.15it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 180842.83it/s]


Removed empty data. Found  91
Removed questions with no pos label. Found  11642
shuffling...


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 296/296 [00:00<00:00, 15853.43it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2708/2708 [00:00<00:00, 8318.22it/s]
Processing text_right with transform: 100%|██████████| 2708/2708 [00:00<00:00, 178815.40it/s]


Removed empty data. Found  8
Removed questions with no pos label. Found  1595
shuffling...


In [6]:
model = ESIM()
model.params['task'] = mz.tasks.Ranking()
model.params['mask_value'] = 0
model.params['input_shapes'] = [[fixed_length_left, ],
                                [fixed_length_right, ]]
model.params['lstm_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = False
model.params['dropout_rate'] = 0.5

model.params['mlp_num_units'] = 300
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = Adam(lr=4e-4)

model.guess_and_fill_missing_params()
model.build()

model.compile()
model.backend.summary() # not visualize

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1930500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)             multiple             0           embedding[0][0]                  
          

In [7]:
# run as classification task
model.load_embedding_matrix(embedding_matrix)
evaluate = mz.callbacks.EvaluateAllMetrics(model,
                                           x=pred_X,
                                           y=pred_Y,
                                           once_every=1,
                                           batch_size=len(pred_Y))

history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],
                    y = train_Y,
                    validation_data = (val_X, val_Y),
                    batch_size = batch_size,
                    epochs = epochs,
                    callbacks=[evaluate]
                    )

Train on 8627 samples, validate on 1130 samples
Epoch 1/5
8627/8627 [==============================] - 48s 6ms/step - loss: 0.1073 - val_loss: 0.0984
Validation: mean_average_precision(0.0): 0.6222655981584554
Epoch 2/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.0994 - val_loss: 0.0974
Validation: mean_average_precision(0.0): 0.640342571890191
Epoch 3/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.0944 - val_loss: 0.0981
Validation: mean_average_precision(0.0): 0.633281742507933
Epoch 4/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.0915 - val_loss: 0.0898
Validation: mean_average_precision(0.0): 0.6479046351993808
Epoch 5/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.0893 - val_loss: 0.0931
Validation: mean_average_precision(0.0): 0.6506805763854636


In [6]:
# run as classification task
classification_task = mz.tasks.Classification(num_classes=2)
classification_task.metrics = 'acc'

model = ESIM()
model.params['task'] = classification_task
model.params['mask_value'] = 0
model.params['input_shapes'] = [[fixed_length_left, ],
                                [fixed_length_right, ]]
model.params['lstm_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = False
model.params['dropout_rate'] = 0.5

model.params['mlp_num_units'] = 300
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = Adam(lr=4e-4)

model.guess_and_fill_missing_params()
model.build()

model.compile()
model.backend.summary() # not visualize

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1930500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)             multiple             0           embedding[0][0]                  
          

In [7]:
evaluate = mz.callbacks.EvaluateAllMetrics(model,
                                           x=pred_X,
                                           y=pred_Y,
                                           once_every=1,
                                           batch_size=len(pred_Y))

train_Y = to_categorical(train_Y)
val_Y = to_categorical(val_Y)

model.load_embedding_matrix(embedding_matrix)
history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],
                    y = train_Y,
                    validation_data = (val_X, val_Y),
                    batch_size = batch_size,
                    epochs = epochs,
                    callbacks=[evaluate]
                    )

Train on 8627 samples, validate on 1130 samples
Epoch 1/5
8627/8627 [==============================] - 48s 6ms/step - loss: 0.3607 - val_loss: 0.3330
Validation: categorical_accuracy: 1.0
Epoch 2/5
8627/8627 [==============================] - 43s 5ms/step - loss: 0.3273 - val_loss: 0.3490
Validation: categorical_accuracy: 0.9451327323913574
Epoch 3/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.3096 - val_loss: 0.3498
Validation: categorical_accuracy: 0.9938052892684937
Epoch 4/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.2970 - val_loss: 0.3170
Validation: categorical_accuracy: 0.969911515712738
Epoch 5/5
8627/8627 [==============================] - 44s 5ms/step - loss: 0.2787 - val_loss: 0.3543
Validation: categorical_accuracy: 0.8778761029243469


In [14]:
model.evaluate(val_X, val_Y)

{'categorical_accuracy': 0.8920354}